# go-interface接口

Go语言里面设计最精妙的应该算interface，它让面向对象，内容组织实现非常的方便.

### 什么是interface?

简单的说，interface是一组method签名的组合，通过interface来定义对象的一组行为。
(注: Go中推荐使用组合,不使用继承来实现代码复用)

### interface类型

* interface类型定义了一组方法，如果某个对象实现了某个接口的所有方法，则此对象就实现了此接口。
* 一个对象可以实现任意多个interface
* interface就是一组抽象方法的集合，它必须由其他非interface类型实现，而不能自我实现， Go通过interface实现了duck-typing.

### interface值

定义了一个interface的变量，则这个变量可以存实现这个interface的任意类型的对象.

```
package main

import "fmt"

type Human struct {
	name string
	age int
	phone string
}

type Student struct {
	Human //匿名字段
	school string
	loan float32
}

type Employee struct {
	Human //匿名字段
	company string
	money float32
}

//Human实现SayHi方法
func (h Human) SayHi() {
	fmt.Printf("Hi, I am %s you can call me on %s\n", h.name, h.phone)
}

//Human实现Sing方法
func (h Human) Sing(lyrics string) {
	fmt.Println("La la la la...", lyrics)
}

//Employee重载Human的SayHi方法
func (e Employee) SayHi() {
	fmt.Printf("Hi, I am %s, I work at %s. Call me on %s\n", e.name,
		e.company, e.phone)
	}

// Interface Men被Human,Student和Employee实现
// 因为这三个类型都实现了这两个方法
type Men interface {
	SayHi()
	Sing(lyrics string)
}

func main() {
	mike := Student{Human{"Mike", 25, "222-222-XXX"}, "MIT", 0.00}
	paul := Student{Human{"Paul", 26, "111-222-XXX"}, "Harvard", 100}
	sam := Employee{Human{"Sam", 36, "444-222-XXX"}, "Golang Inc.", 1000}
	tom := Employee{Human{"Tom", 37, "222-444-XXX"}, "Things Ltd.", 5000}

	//定义Men类型的变量i
	var i Men

	//i能存储Student
	i = mike
	fmt.Println("This is Mike, a Student:")
	i.SayHi()
	i.Sing("November rain")

	//i也能存储Employee
	i = tom
	fmt.Println("This is tom, an Employee:")
	i.SayHi()
	i.Sing("Born to be wild")

	//定义了slice Men
	fmt.Println("Let's use a slice of Men and see what happens")
	x := make([]Men, 3)
	//这三个都是不同类型的元素，但是他们实现了interface同一个接口
	x[0], x[1], x[2] = paul, sam, mike

	for _, value := range x{
		value.SayHi()
	}
}
```

* 接口是用来定义行为的类型。这些被定义的行为不由接口直接实现，而是通过方法由用户定义的类型实现。
* 如果用户定义的类型实现了某个接口类型声明的一组方法，那么这个用户定义的类型的值就可以赋给这个接口类型的值。这个赋值会把用户定义的类型的值存入接口类型的值。
* 对接口值方法的调用会执行接口值里存储的用户定义的类型的值对应的方法。

* 接口值是一个两个字长度的数据结构
    * 第一个字包含一个指向内部表的指针。这个内部表叫作 iTable，包含了所存储的值的类型信息。iTable 包含了已存储的值的类型信息以及与这个值相关联的一组方法。
    * 第二个字是一个指向所存储值的指针。将类型信息和指针组合在一起，就将这两个值组成了一种特殊的关系。

* Go 语言提供了另外一种数据类型: 接口

* 接口定义: 把所有的具有共性的方法定义在一起，任何其他类型只要实现了这些方法就是实现了这个接口。

```
/* 定义接口 */
type interface_name interface {
   method_name1 [return_type]
   method_name2 [return_type]
   method_name3 [return_type]
   ...
   method_namen [return_type]
}

/* 定义结构体 */
type struct_name struct {
   /* variables */
}

/* 实现接口方法 */
func (struct_name_variable struct_name) method_name1() [return_type] {
   /* 方法实现 */
}
...
func (struct_name_variable struct_name) method_namen() [return_type] {
   /* 方法实现*/
}
```

### 接口实现的例子

```go
package main

import (
    "fmt"
)

type Phone interface {
    call()
}

type NokiaPhone struct {
}

func (nokiaPhone NokiaPhone) call() {
    fmt.Println("I am Nokia, I can call you!")
}

type IPhone struct {
}

func (iPhone IPhone) call() {
    fmt.Println("I am iPhone, I can call you!")
}

func main() {
    var phone Phone

    phone = new(NokiaPhone)
    phone.call()

    phone = new(IPhone)
    phone.call()

}
```

### 空interface

空interface(interface{})不包含任何的method，正因为如此，所有的类型都实现了空interface。空interface对于描述起不到任何的作用(因为它不包含任何的method），但是空interface在我们需要存储任意类型的数值的时候相当有用，因为它可以存储任意类型的数值。它有点类似于C语言的`void*`类型。

```
// 定义a为空接口
var a interface{}
var i int = 5
s := "Hello world"
// a可以存储任意类型的数值
a = i
a = s
```
一个函数把interface{}作为参数，那么它可以接受任意类型的值作为参数，如果一个函数返回interface{},也就可以返回任意类型的值。是不是很有用啊！

### interface函数参数

interface的变量可以持有任意实现该interface类型的对象, 可以通过定义interface参数，让函数接受各种类型的参数。

### interface变量存储的类型

interface的变量里面可以存储任意类型的数值(该类型实现了interface)。那么如何反向知道这个变量里面实际保存了哪些类型的对象呢？

目前常用有两种方法：
* Comma-ok断言: Go语言里面有一个语法，可直接判断是否是该类型的变量：`value, ok = element.(T)`，这里value是变量的值，ok是bool类型，element是interface变量，T是断言的类型。 如果element里面确实存储了T类型的数值，那么ok返回true，否则返回false。
* switch测试:


### 嵌入interface

Go里面真正吸引人的是它内置的逻辑语法, 如果一个interface1作为interface2的一个嵌入字段，那么interface2隐式的包含了interface1里面的method。

### 反射

Go语言实现了反射，所谓反射就是能检查程序在运行时的状态。我们一般用到的包是reflect包。

反射的字段必须是可修改的，即反射的字段必须是可读写的.

### 嵌入类型(组合)

Go 语言允许用户扩展或者修改已有类型的行为。这个功能对代码复用很重要，这个就是嵌入类型（type embedding）。

嵌入类型是将已有的类型直接声明在新的结构类型里。被嵌入的类型被称为新的外部类型的内部类型。

通过嵌入类型，与内部类型相关的标识符会提升到外部类型上。这些被提升的标识符就像直接声明在外部类型里的标识符一样，也是外部类型的一部分。这样外部类型就组合了内部类型包含的所有属性，并且可以添加新的字段和方法。外部类型也可以通过声明与内部类型标识符同名的标识符来覆盖内部标识符的字段或者方法。这就是扩展或者修改已有类型的方法。